%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [35]:
from IMLearn.learners import UnivariateGaussian  # TODO resolve import (with\without IMLearn)
import numpy as np
import plotly.express as px

In [36]:
def univariate_gaussian_estimation():
    # Q1:
    mu, var = 10, 1
    samples = np.random.normal(loc=mu, scale=var, size=1000)

    estimator = UnivariateGaussian().fit(samples)
    print((estimator.mu_, estimator.var_))

    # Q2:
    expectations = [abs(mu - UnivariateGaussian().fit(samples[:n]).mu_) for n in range(10, 1010, 10)]
    fig_1 = px.scatter(x=[n for n in range(10, 1010, 10)], y=expectations,
                    labels={
                        "x": "sample size",
                        "y": "absolute distance"})
    
    _fig_1_config(fig_1)
    fig_1.show()
    
    # Q3:
    # I'm expecting to see a normal (Gaussian) distribution around 10, with a variance of 1
    pdfs = estimator.pdf(samples)
    X = np.column_stack((samples, pdfs))
    X = X[np.argsort(X[:, 1])]
    
    fig_2 = px.scatter(x=X[:, 0], y=X[:, 1],
                      labels={
                          "x": "sample",
                          "y": "empirical PDF"})
    _fig_2_config(fig_2)
    fig_2.show()

In [37]:
def _fig_1_config(fig):
    fig.update_layout(
        title={
            "text": "Error of Estimated Value of Expectation, as a Function of Sample Size",
            "x": 0.5, 
            "y": 0.95},
        template="simple_white",
        font_color="black",
        title_font_family="Helvetica",
        title_font_color="black",
    )
    fig.update_traces(
        marker=dict(size=5, symbol="diamond", color="black"),
        selector=dict(mode="markers"),
    )
    fig.update_xaxes(
        title="Sample Size",
        title_font_family="Times New Roman", tickmode = "linear", tick0 = 0, dtick = 50)
    fig.update_yaxes(
        title="Absolute distance between est. and true value of expectation",
        title_font_family="Times New Roman")

def _fig_2_config(fig):
    fig.update_layout(
        title={
            "text": "Empirical PDF Under the Fitted Model",
            "x": 0.5, 
            "y": 0.95},
        template="simple_white",
        font_color="black",
        title_font_family="Helvetica",
        title_font_color="black")
    fig.update_traces(
        marker=dict(size=5, symbol="diamond", color="black"),
        selector=dict(mode="markers"))
    fig.update_xaxes(
        title="x (sample)",
        title_font_family="Times New Roman")
    fig.update_yaxes(
        title="Empirical PDF",
        title_font_family="Times New Roman")

In [38]:
def multivariate_gaussian_estimation():
    # Q4:
    pass

In [39]:
if __name__ == '__main__':
    np.random.seed(173)
    univariate_gaussian_estimation()
    multivariate_gaussian_estimation()

(10.000348480377385, 0.9485592794250123)
